## 6.2 Static arrays

A **static array** is a data structure that stores a fixed-length sequence
contiguously in memory, with the same number of bytes per item. This allows us
to compute the address of any item, and thereby access it, in constant time.
For example, the next figure shows a static array storing the string 'Hi!'
from memory address 123 onwards, using two bytes per character.

![The figure shows three rectangular boxes next to each other.
The left-most box has numbers 123 and 124 over it and the letter H inside it.
The middle box has numbers 125 and 126 over it and the letter i inside it.
The right-most box has numbers 127 and 128 over it and an exclamation mark
inside it.](06_2_sa1.png)

The character at index _i_ is stored at the address 123 + _i_ × 2 and the next address,
e.g. the character at index&nbsp;1&nbsp;starts at address 123 + 1 × 2 = 125.
(Remember that indices start at zero.)
More generally, if an array at memory _start_ allocates _n_ bytes per item,
the item at index _i_ can be found at address _start_ + _i_ × _n_ and the
subsequent _n_ − 1&nbsp;bytes.

<div class="alert alert-info">
<strong>Info:</strong> The term 'array' can mean different things (static array, list,
homogeneous sequence, etc.), depending on the author or programming language.
To avoid confusions, we use 'static array' in M269, with the above meaning.
</div>

Static arrays are the ideal data structure for immutable sequences where all
items occupy the same number of bytes, like strings and tuples of Booleans.
But how to store a tuple of strings of different lengths,
or a heterogeneous tuple? The trick is to store in the array
the addresses of the items instead of the items themselves.
Let's suppose we have $2^{32}$ bytes = 4&nbsp;GB of RAM.
We can represent any memory location as a 32-bit integer.
The next figure illustrates the storage of tuple `('Hi!', 'Go', 'left')`.
The static array for the tuple contains the address of each string.
The strings may be anywhere in memory,
not necessarily next to the tuple or each other.

![The figure shows four rows of boxes.
The first row has three boxes containing numbers 123, 100542 and 1423208.
These are the first addresses of the other rows of boxes. Each first row box
occupies 4&nbsp;bytes, indicated by the addresses above the boxes
going from 1050 to 1053 for the left-hand box, 1054 to 1057 for the middle box
and 1058 to 1061 for the right-hand box.
The second row of boxes are the same as in the previous figure:
three boxes with uppercase H, lowercase i and an exclamation mark inside them,
and addresses 123 to 128 over them.
The third row has two boxes, with uppercase G and lowercase o inside them,
and addresses 100542 to 100545 above them.
The fourth row has four boxes, with lowercase l, e, f and t inside,
and addresses 1423208 to 1423215 above them.](06_2_sa2.png)

We usually don't know (and don't care) where in memory a data structure
will be stored. It's also often irrelevant how many bytes characters,
addresses and other items occupy. Abstracting away those details,
we can represent the tuple and its three strings in a more schematic way.

![The figure shows four separate groups of boxes.
Three of the groups contain the words Hi!, Go and left,
with one character per box.
The fourth group has three boxes, each containing the start of an arrow.
Each of the three arrows ends below the first box of one of the other three groups.
The arrow starting in the left-hand box points to H (the first box of Hi!),
the arrow starting in the middle box points to G (the first box of Go) and
the arrow starting in the right-hand box points to l
(the first box of left).](06_2_sa3.png)

This image conveys the gist of the data structure:
a string is stored as a static array of characters, and the tuple of strings is
stored as a static array of references to those strings.
A **reference** is an object that refers to another object.
If the reference consists of the memory address of that other object,
we call it a **pointer**.

Before continuing with static arrays, let me illustrate how the use of
references explains the behaviour of assignments in Python.

### 6.2.1 Variables and assignments

There are other kinds of reference objects.
An index is an integer object that refers to a particular item of a sequence.
A variable is a named reference to an object: you can think of it as
a string-pointer pair with the name of the variable
and the address of the object it refers to.

Consider the following example, which uses a list instead of a tuple.

In [1]:
x = 1           # the x-y coordinates of a 2D point
y = 2
pair = [x, y]
point = pair

Assuming that lists are also stored as static arrays,
the situation in memory at this stage is as follows:

![The figure shows variable names pair, point, x and y.
Names pair and point have arrows next to them leading to a pair of boxes.
The first box has an arrow that leads to a separate box, with number 1 in it.
The second box has an arrow that leads to a separate box, with number 2 in it.
Variable name x has an arrow that leads to the box with the number 1.
Variable y has an arrow that leads to the box with the number 2.](06_2_va1.png)

Names `pair` and `point` refer to exactly the same list object,
`pair[0]` and `x` refer to the exact same integer object with value one,
and `pair[1]` and `y` refer to the same integer object with value two.
Let's change the value that `y` refers to:

In [2]:
y = 3

The Python interpreter does _not_ change the value of the existing integer
object. Instead, it creates a new integer object with value three,
and makes the variable refer to that object.
In Python, `variable = value` 'attaches' the variable's name to the value,
rather than modifies the current variable's value.
The data structures look now like this:

![This figure is the same as the previous one, but the arrow next to
variable y is no longer pointing at the box with number 2. Instead, it's
pointing at a new box with number 3.](06_2_va2.png)

Hence, if we display the list, it hasn't changed:

In [3]:
point

[1, 2]

If we do this assignment,

In [4]:
pair[1] = 3

then the interpreter again creates an integer object with value 3 and
makes the name `pair[1]` refer to that object. The situation is now:

![This figure is the same as the previous one, but the arrow stemming from
the second of the pair of boxes is no longer pointing at the box with number 2.
Instead, it's pointing at yet another new box with number 3,
so there are two boxes with number 3: one has an arrow from variable y,
the other has an arrow from the second of the pair of boxes.
The box with number 2&nbsp;has no arrows leading to it.](06_2_va3.png)

Let's display again the point's coordinates:

In [5]:
point

[1, 3]

They have changed, even though the assignment was to `pair[1]`, not `point[1]`.
The earlier assignment `point = pair` created a new reference to the same list,
it didn't create a copy of the list.

<div class="alert alert-info">
<strong>Info:</strong> TM112 Block&nbsp;2 Sections 4.5.1 to 4.5.3 explain the above in more detail.
</div>

The diagram also shows that the integer object with value two isn't referred by
any other object anymore. The Python interpreter detects that automatically,
and frees the memory occupied by the object so that it can be reused later.
This process is known as **garbage collection**.

### 6.2.2 The `StaticArray` class

Some languages provide a static array data type to implement a restricted form
of the sequence ADT, namely fixed-length homogeneous sequences.
Besides the length of the array, one has to indicate the type of items.
This allows the interpreter to figure out how many bytes per item are needed
and the initial value of each item, to then create the static array in memory.
For example, an array of Booleans may be created with one byte per Boolean,
each initialised to false, whereas an array of strings may be created with
pointers to the same empty string.

<div class="alert alert-info">
<strong>Info:</strong> Java has a static array data type, simply called array,
with indices starting at zero.
The type of items is indicated when the array is declared, and
the length is determined when the array is initialised.
Java arrays can be heterogeneous if the item's type is <code>Object</code>.
</div>

A data type for static arrays only has to support three operations,
besides the constructor: length, indexing, and replacing an item.
All other operations, e.g. slicing, can be implemented with these basic operations.

Python doesn't include such a data type as it already has a far more flexible
one: lists. The implementation of lists may differ between Python interpreters,
because the Python language only defines the list as an ADT,
i.e. which operations it supports.
However, it's safe to assume all interpreters use static arrays as the
underlying data structure in order to achieve constant-time indexing.
We can thus use lists as if they were static arrays providing we never call methods
`append`, `insert` and `pop`. I will however define a new data type,
to make the use of static arrays explicit in subsequent examples.

I won't restrict my static array data type to any particular type of items.
The constructor simply takes the length of the array,
and initialises it with `None` in every position.
The essential operations on an array are to obtain its length, and to
access and replace an item at a given index.
Following my definition of the
[indexing operation](../04_Iteration/04_1_sequences.ipynb#4.1.1-Inspecting-sequences),
I assume indices aren't negative.
For convenience I also implement an operation to convert to a string.

I write the class to a file to reuse it later.

In [6]:
# this code is also in m269_array.py

class StaticArray:
    """A fixed-length sequence of references in contiguous memory."""

    def __init__(self, length: int):
        """Create an array of the given length.

        Preconditions: length >= 0
        Postconditions: every item in the array is None
        """
        # assume lists are stored in arrays
        self.items = [None] * length

    def length(self) -> int:
        """Return the length of the array."""
        return len(self.items)

    def get_item(self, index: int) -> object:
        """Return the item at the given index.

        Preconditions: 0 <= index < self.length()
        """
        return self.items[index]

    def set_item(self, index: int, item: object) -> None:
        """Replace the item at the given index with the given item.

        Preconditions: 0 <= index < self.length()
        Postconditions: self.get_item(index) == item
        """
        self.items[index] = item

    def __str__(self) -> str:
        """Return a string representation of the array."""
        return str(self.items)

In Python, every object is an instance of a class appropriately named `object`.
All other classes are subclasses of `object`.
Class _A_ is a **subclass** of class _B_, and _B_ is a **superclass** of _A_,
if every instance of _A_ is also an instance of _B_.
Using `object` in a method or function header indicates that the input or
output value can be any object. As such, the method or function body can only
use the equality and inequality comparisons or assign the object to a variable.

### 6.2.3 Testing methods

Our [generic test function](../04_Iteration/04_7_reverse.ipynb#4.7.6-Tests) is meant for testing functions
with a single output that can be easily compared against an expected value.
However, class methods sometimes modify an instance variable instead of
returning a value, like `set_item` above. We need a different approach.

**White-box tests**, also known as structural tests,
have full knowledge of the data structure(s) used by the data type.
Structural tests can be written independently for each method, as they can
access the instance variables before and after the method to check
they were correctly modified. The disadvantage of structural tests is that
they must be rewritten whenever the data structures change.

**Black-box tests**, also known as specification-based tests,
don't access the data structures.
They test methods based on their defined behaviour.
This means that methods can't be tested independently: a method that
changes data can only be tested with methods that inspect data.
Black-box testing makes **regression testing** easier.
That's the process of running the same tests after each change,
to make sure the modified code still behaves as before.

For example, a white-box test of the `__init__` method can iterate over
the `self.items` list to check all items were initialised to `None`,
whereas a black-box test has to use method `get_item`.

In M269 we'll use mostly black-box testing: a single set of tests,
based on the ADT's operations, can be run repeatedly to test
different implementations of the same ADT. This saves work and time.

To make test code shorter and more readable, I define an auxiliary function
for the error messages and write it to a file to reuse it later.

In [7]:
# this code is also in m269_check.py

def check(case: str, actual: object, expected: object, context: object) -> None:
    """Write a message if actual and expected are different."""
    if actual != expected:
        print(case, 'FAILED for', context, ':', actual, 'instead of', expected)

The context is some additional information to help us make sense of the error.
In the following, the context is the static array for which the test failed.

Let's start by testing the initialisation method for an arbitrary array length.
We must check it has created the right number of items, all of them `None`.

In [8]:
def test_init(length: int) -> None:
    """Create a new array of the given length and check it.

    Preconditions: length >= 0
    """
    array = StaticArray(length)
    check('length', array.length(), length, array)
    for index in range(0, length):
        check('initial item', array.get_item(index), None, array)

The black-box test uses methods `length` and `get_item` to inspect
the created instance.
As such, we're also testing these methods besides `__init__`.

We also test `set_item` by creating an array and checking it with `get_item`.
I often use the sequence 0, 1, 2, ... for testing, because if a test fails
it's easy to see which value is not at the right index.

<div class="alert alert-warning">
<strong>Note:</strong> When testing operations on sequences, use sequences of the form (0, 1, 2, ...).
</div>

In [9]:
def test_set_item(length: int) -> None:
    """Create an array of the given length, replace all items and check it.

    Preconditions: length >= 0
    """
    array = StaticArray(length)
    for index in range(length):
        array.set_item(index, index)
    for index in range(length):
        check('replaced item', array.get_item(index), index, array)

This only tests `set_item` with a homogeneous array.
In general, you may need several test functions for one method.
It's not unusual for test code to be longer than the code it tests.

I can now test arrays of different lengths.
The `check` function will print every array that fails a test.

In [10]:
test_init(0)    # edge case: length zero
for length in range(1, 11):
    test_init(length)
    test_set_item(length)

In testing, no news is good news.

<div class="alert alert-warning">
<strong>Note:</strong> To black-box test a class, write one or more test functions for each
method that creates or modifies an instance of that class,
using the methods that inspect the instance.
Together, the tests must use all inspection methods.
</div>

⟵ [Previous section](06_1_define_type.ipynb) | [Up](06-introduction.ipynb) | [Next section](06_3_develop_type.ipynb) ⟶